<a href="https://colab.research.google.com/github/asurm22/MLass4/blob/main/imporved_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!pip install torch torchvision torchaudio
!pip install wandb
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

In [ ]:
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip -d data/

In [ ]:
!ls data

In [ ]:
!pip install -q wandb

import wandb
wandb.login()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('data/icml_face_data.csv')

In [ ]:
train_df.head()

In [ ]:
train = train_df[train_df[' Usage'] == 'Training']
validation = train_df[train_df[' Usage'] == 'PrivateTest']
test = train_df[train_df[' Usage'] == 'PublicTest']

In [ ]:
print(test.columns)

In [ ]:
def preprocess(df):
    pixels = df[' pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.float32).reshape(48, 48))
    images = np.stack(pixels.values)
    labels = df['emotion'].values
    return images, labels

X_train, y_train = preprocess(train)
X_val, y_val = preprocess(validation)
X_test, y_test = preprocess(test)

In [ ]:
wandb.init(
    project="fer-challenge",
    name="improved-cnn-bs",
    config={
        "epochs": 40,
        "batch_size": 64,
        "learning_rate": 1e-3,
        "architecture": "BetterCNN",
        "dataset": "FER2013",
        "weight_decay": 1e-4,
        "label_smoothing": 0.1,
        "patience": 5
    }
)
config = wandb.config

In [ ]:
from torchvision.transforms import RandomErasing

transform_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomCrop(48, padding=4),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3))
])

transform_eval = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

def preprocess(df):
    pix = df[' pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.uint8).reshape(48,48))
    return np.stack(pix.values), df['emotion'].astype(int).values

X_train, y_train = preprocess(train)
X_val,   y_val   = preprocess(validation)
X_test,  y_test  = preprocess(test)

class FERDataset(Dataset):
    def __init__(self, images, labels, transform):
        self.images = images
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        img = self.images[idx]
        lbl = self.labels[idx]
        img = self.transform(img)
        return img, lbl

train_ds = FERDataset(X_train, y_train, transform_train)
val_ds   = FERDataset(X_val,   y_val,   transform_eval)
test_ds  = FERDataset(X_test,  y_test,  transform_eval)

train_loader = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True,  num_workers=2)
val_loader   = DataLoader(val_ds,   batch_size=config.batch_size, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds,  batch_size=config.batch_size, shuffle=False, num_workers=2)

In [ ]:
 # class LabelSmoothingCrossEntropy(nn.Module):
    #def __init__(self, smoothing=0.1):
        #super().__init__()
        #self.smoothing = smoothing
       # self.confidence = 1.0 - smoothing
   # def forward(self, pred, target):
        #log_probs = torch.nn.functional.log_softmax(pred, dim=-1)
       # true_dist = torch.zeros_like(log_probs)
       # true_dist.fill_(self.smoothing / (pred.size(1) - 1))
       # true_dist.scatter_(1, target.unsqueeze(1), self.confidence)
       # return torch.mean(torch.sum(-true_dist * log_probs, dim=-1))

class BetterCNN(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(1, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2), nn.Dropout(0.25),

            # Block 2
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128,128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2), nn.Dropout(0.25),

            # Block 3 (two convs instead of one)
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.MaxPool2d(2), nn.Dropout(0.25)
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((6,6)),  # avoids magic flatten size
            nn.Flatten(),
            nn.Linear(256*6*6, 512), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


In [51]:
from sklearn.metrics import confusion_matrix
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BetterCNN().to(device)
wandb.watch(model, log="all")

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=1e-2,
    steps_per_epoch=len(train_loader),
    epochs=config.epochs,
    pct_start=0.3,
    div_factor=10,
    final_div_factor=100
)

best_val_acc = 0.0
epochs_no_improve = 0

for epoch in range(1, config.epochs+1):
    # ——— Training ———
    model.train()
    running_loss = 0.0
    correct = 0
    for imgs, labs in train_loader:
        imgs, labs = imgs.to(device), labs.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labs)
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
        correct += (outputs.argmax(1)==labs).sum().item()
    train_loss = running_loss/len(train_loader)
    train_acc  = correct/len(train_loader.dataset)

    # ——— Validation ———
    model.eval()
    correct = 0
    val_loss = 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labs in val_loader:
            imgs, labs = imgs.to(device), labs.to(device)
            outs = model(imgs)
            val_loss += criterion(outs, labs).item()
            preds = outs.argmax(1)
            correct += (preds==labs).sum().item()
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labs.cpu().tolist())
    val_loss /= len(val_loader)
    val_acc  = correct/len(val_loader.dataset)

    # ——— Scheduler & Early Stop ———
    scheduler.step(val_acc)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pt")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= config.patience:
            print(f"Early stopping at epoch {epoch}")
            break

    # ——— wandb Logging ———
    cm = confusion_matrix(all_labels, all_preds)
    wandb.log({
        "epoch": epoch,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc,
        "lr": optimizer.param_groups[0]['lr'],
        "conf_mat": wandb.plot.confusion_matrix(probs=None,
                                                y_true=all_labels,
                                                preds=all_preds,
                                                class_names=[str(i) for i in range(7)])
    })

    print(f"Epoch {epoch}/{config.epochs} — "
          f"Train: {train_loss:.4f}, {train_acc:.4f}  |  "
          f"Val:   {val_loss:.4f}, {val_acc:.4f}")

# ——— Final Test Eval ———
model.load_state_dict(torch.load("best_model.pt"))
model.eval()
correct = 0
with torch.no_grad():
    for imgs, labs in test_loader:
        imgs, labs = imgs.to(device), labs.to(device)
        preds = model(imgs).argmax(1)
        correct += (preds==labs).sum().item()
test_acc = correct/len(test_loader.dataset)
print(f"Test Accuracy: {test_acc:.4f}")
wandb.log({"test_accuracy": test_acc})
wandb.finish()


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 1/40 — Train: 1.9584, 0.2370  |  Val:   1.8430, 0.2396


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 2/40 — Train: 1.8528, 0.2485  |  Val:   1.8408, 0.2449


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


...

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 28/40 — Train: 1.6673, 0.3313  |  Val:   1.5183, 0.4606
Early stopping at epoch 29


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Test Accuracy: 0.4572


epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁███████████████
test_accuracy,▁
train_acc,▁▂▂▂▂▂▂▂▂▄▄▅▅▅▆▆▇▇█▇▇▇██████
train_loss,█▅▅▅▅▅▅▅▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▁▁▁▁▁▁▂▃▃▄▄▇▇▇▇█▇█▇███████
val_loss,█████▇▇▇▅▅▄▃▃▃▂▃▃▂▂▂▂▂▁▁▂▁▁▁
epoch,28
lr,0.001
test_accuracy,0.45723
train_acc,0.33129
